In [1]:
import gym
import sys
sys.path.append('./src')

#env_id = "ALE/Pong-v5"
env_id = 'CartPole-v1'
# Create the env
env = gym.make(env_id)

env.reset()


array([ 0.00446986, -0.02437763, -0.0368451 , -0.01882935], dtype=float32)

In [2]:
from feedback import FeedbackManager

fb = FeedbackManager("testcartpole.dat")

Feedback loaded from testcartpole.dat


In [3]:
import torch
device = torch.device("cuda")

In [4]:
from rewardmodel import RewardModel

rewardmodel = RewardModel(env.observation_space.shape, device)

Load: False


In [5]:
for i in range(1000):
    batch = fb.randomBatch(20)
    rewardmodel.train(batch)

Reward model loss: 13.669962882995605
Reward model loss: 13.671796798706055
Reward model loss: 13.751520156860352
Reward model loss: 13.193249702453613
Reward model loss: 13.144682884216309
Reward model loss: 13.00345230102539
Reward model loss: 13.038810729980469
Reward model loss: 13.003327369689941
Reward model loss: 13.289158821105957
Reward model loss: 12.824932098388672
Reward model loss: 13.037227630615234
Reward model loss: 12.660341262817383
Reward model loss: 12.876434326171875
Reward model loss: 12.650230407714844
Reward model loss: 12.474855422973633
Reward model loss: 12.996648788452148
Reward model loss: 12.793920516967773
Reward model loss: 12.337895393371582
Reward model loss: 12.267553329467773
Reward model loss: 11.993328094482422
Reward model loss: 12.064966201782227
Reward model loss: 12.604612350463867
Reward model loss: 12.221847534179688
Reward model loss: 11.852453231811523
Reward model loss: 11.947577476501465
Reward model loss: 12.244416236877441
Reward model 

Reward model loss: 8.645771026611328
Reward model loss: 8.119784355163574
Reward model loss: 8.561381340026855
Reward model loss: 9.174245834350586
Reward model loss: 8.700199127197266
Reward model loss: 8.536866188049316
Reward model loss: 10.196405410766602
Reward model loss: 10.350835800170898
Reward model loss: 9.488889694213867
Reward model loss: 8.6554594039917
Reward model loss: 7.783965110778809
Reward model loss: 7.612686634063721
Reward model loss: 8.998015403747559
Reward model loss: 9.090738296508789
Reward model loss: 8.52303695678711
Reward model loss: 7.734999179840088
Reward model loss: 9.416711807250977
Reward model loss: 9.1477632522583
Reward model loss: 8.678762435913086
Reward model loss: 8.650921821594238
Reward model loss: 9.477124214172363
Reward model loss: 9.565399169921875
Reward model loss: 9.17266845703125
Reward model loss: 8.871453285217285
Reward model loss: 8.9016695022583
Reward model loss: 7.94265079498291
Reward model loss: 8.343598365783691
Reward m

Reward model loss: 7.819600582122803
Reward model loss: 7.865680694580078
Reward model loss: 7.8941216468811035
Reward model loss: 8.481367111206055
Reward model loss: 8.632038116455078
Reward model loss: 7.677498817443848
Reward model loss: 8.699337005615234
Reward model loss: 7.477108001708984
Reward model loss: 7.488408088684082
Reward model loss: 8.354462623596191
Reward model loss: 6.454158782958984
Reward model loss: 7.849917411804199
Reward model loss: 8.199522018432617
Reward model loss: 7.0104594230651855
Reward model loss: 7.837678909301758
Reward model loss: 7.951850414276123
Reward model loss: 7.95158576965332
Reward model loss: 7.3054914474487305
Reward model loss: 9.099222183227539
Reward model loss: 9.032134056091309
Reward model loss: 7.259716987609863
Reward model loss: 7.394374847412109
Reward model loss: 8.736743927001953
Reward model loss: 8.019903182983398
Reward model loss: 7.8425822257995605
Reward model loss: 7.334686279296875
Reward model loss: 8.47293949127197

Reward model loss: 7.694941520690918
Reward model loss: 8.318954467773438
Reward model loss: 7.661291599273682
Reward model loss: 7.785346984863281
Reward model loss: 8.142732620239258
Reward model loss: 7.361728668212891
Reward model loss: 8.079889297485352
Reward model loss: 8.460212707519531
Reward model loss: 7.702698707580566
Reward model loss: 8.191869735717773
Reward model loss: 6.959529399871826
Reward model loss: 7.686008930206299
Reward model loss: 7.65496826171875
Reward model loss: 7.732361793518066
Reward model loss: 8.312554359436035
Reward model loss: 7.495898246765137
Reward model loss: 8.287330627441406
Reward model loss: 8.181438446044922
Reward model loss: 9.13190746307373
Reward model loss: 7.543603897094727
Reward model loss: 7.6306962966918945
Reward model loss: 7.591620922088623
Reward model loss: 8.282498359680176
Reward model loss: 7.350874423980713
Reward model loss: 6.9546284675598145
Reward model loss: 7.389833927154541
Reward model loss: 7.073119163513184
R

Reward model loss: 8.075644493103027
Reward model loss: 8.111801147460938
Reward model loss: 8.202411651611328
Reward model loss: 7.958775997161865
Reward model loss: 8.050945281982422
Reward model loss: 6.066075325012207
Reward model loss: 7.29002046585083
Reward model loss: 7.379780292510986
Reward model loss: 8.238798141479492
Reward model loss: 8.092264175415039
Reward model loss: 7.42347526550293
Reward model loss: 8.124956130981445
Reward model loss: 7.520751953125
Reward model loss: 7.572120666503906
Reward model loss: 8.081446647644043
Reward model loss: 7.476131439208984
Reward model loss: 8.275296211242676
Reward model loss: 6.590890884399414
Reward model loss: 6.545436859130859
Reward model loss: 8.73849105834961
Reward model loss: 7.4232988357543945
Reward model loss: 7.3488545417785645
Reward model loss: 7.535915851593018
Reward model loss: 7.568361759185791
Reward model loss: 6.634130477905273
Reward model loss: 7.485108852386475
Reward model loss: 6.790674209594727
Rewar

In [103]:
env.reset()

array([ 0.00999877,  0.03353493,  0.01571354, -0.0032482 ], dtype=float32)

In [192]:
state, _, done, _ = env.step(env.action_space.sample())
env.render()
if done:
    env.reset()
rewardmodel.evaluate(state)

tensor([0.5894], device='cuda:0')